# Variation du salaire de base

Initialisations Python

In [ ]:
from datetime import date
import matplotlib.pyplot as plt
#importer numpy permet de coler deux sections de courbes (nécessaire quand on fait varier le salaire de base en-dessous du SMIC mensuel)
import numpy as np
import openfisca_france
from openfisca_core import tools
%matplotlib inline

Initialisations OpenFisca

In [ ]:
tax_benefit_system = openfisca_france.FranceTaxBenefitSystem()

In [ ]:
date_naissance_parent1 = date(1980, 1, 1)

In [ ]:
nb_heures_par_mois_temps_plein = 35 * 52 / 12.0
smic_horaire = 9.670

In [ ]:
# On commence par simuler la variation de salaire entre 0 € et le SMIC mensuel
# Entre 0 € et le SMIC mensuel, quand le salaire de base augmente, c'est le volume horaire qui augmente (de 0 à 35 heures hendomadaires)
# Le salaire horaire quant à lui reste fixé au SMIC horaire

scenario_under_smic = tax_benefit_system.new_scenario().init_single_entity(
    axes = [
        [
            dict(
                count = 10,
                min = 0,
                max = nb_heures_par_mois_temps_plein,
                name = 'heures_remunerees_volume',
                ),
            dict(
                count = 10,
                min = 0,
                max = nb_heures_par_mois_temps_plein * smic_horaire,
                name = 'salaire_de_base',
                ),
            ],
        ],
    period = '2016-01',
    parent1 = dict(
        date_naissance = date_naissance_parent1,
        contrat_de_travail = 1, # temps partiel
        allegement_fillon_mode_recouvrement = 1, # anticipe_regularisation_fin_de_periode
        ),
    )

In [ ]:
simulation_under_smic = scenario_under_smic.new_simulation()

In [ ]:
salaire_de_base_under_smic = simulation_under_smic.calculate('salaire_de_base', '2016-01')
salaire_de_base_under_smic

In [ ]:
allegement_fillon_under_smic = simulation_under_smic.calculate('allegement_fillon', '2016-01')
allegement_fillon_under_smic

In [ ]:
plt.plot(salaire_de_base_under_smic, allegement_fillon_under_smic)
plt.xlabel(u'Salaire de base (€/mois)')
plt.ylabel(u'Allègement Fillon (€/mois)')
plt.title(u'Au-dessous du SMIC')
plt.grid(True)
plt.show()

In [ ]:
smic_mensuel = nb_heures_par_mois_temps_plein * smic_horaire
smic_mensuel

In [ ]:
# Au-delà du SMIC mensuel, on suppose que le volume horaire reste fixé à 35 heures,
# et que c'est le salaire horaire qui augmente

scenario_over_smic = tax_benefit_system.new_scenario().init_single_entity(
    axes = [
        dict(
            count = 50,
            min = smic_mensuel,
            max = 5000,
            name = 'salaire_de_base',
            ),
        ],
    period = '2016-01',
    parent1 = dict(
        date_naissance = date_naissance_parent1,
        contrat_de_travail = 0, # temps plein
        allegement_fillon_mode_recouvrement = 1, # anticipe_regularisation_fin_de_periode
        ),
    )

In [ ]:
simulation_over_smic = scenario_over_smic.new_simulation()

In [ ]:
salaire_de_base_over_smic = simulation_over_smic.calculate('salaire_de_base', '2016-01')
salaire_de_base_over_smic

In [ ]:
allegement_fillon_over_smic = simulation_over_smic.calculate('allegement_fillon', '2016-01')
allegement_fillon_over_smic

In [ ]:
plt.plot(salaire_de_base_over_smic, allegement_fillon_over_smic)
plt.xlabel(u'Salaire de base (€/mois)')
plt.ylabel(u'Allègement Fillon (€/mois)')
plt.title(u'Au-dessus du SMIC')
plt.grid(True)
plt.show()

In [ ]:
# On crée ici la variable "salaire_de_base" qui est l'enchainement du segment : 0 €- SMIC mensuel et du segment SMIC mensuel - au-delà
# le crochet [:-1] permet d'éviter que l'on ait l'observation "SMIC mensuel" en doublon

salaire_de_base = np.concatenate([salaire_de_base_under_smic[:-1], salaire_de_base_over_smic])

In [ ]:
allegement_fillon = np.concatenate([allegement_fillon_under_smic[:-1], allegement_fillon_over_smic])

In [ ]:
plt.plot(salaire_de_base, allegement_fillon)
# plt.plot(salaire_de_base_under_smic, allegement_fillon_under_smic, label=u'Temps partiel de 0 à 35h/mois')
# plt.plot(salaire_de_base_over_smic, allegement_fillon_over_smic, label=u'Temps plein')
plt.xlabel(u'Salaire de base (€/mois)')
plt.ylabel(u'Allègement Fillon (€/mois)')
plt.title(u'Variation du salaire de base mensuel avec réduction du volume horaire au-dessous du SMIC')
plt.grid(True)
plt.axvline(x=smic_mensuel, color='r', label='SMIC')
plt.legend()
plt.show()